<font size='5'> Superstore Sales Analysis </font>

In [1]:
import sqlite3
import pandas as pd 
conn = sqlite3.connect("superstore.db")

<font size ='4' > Examine Superstore Schema </font>

In [6]:
query = "SELECT name FROM sqlite_master WHERE type='table';"
pd.read_sql_query(query, conn)

,name
0,superstore


In [7]:
query = "PRAGMA table_info(superstore);"
pd.read_sql_query(query, conn)

,cid,name,type,notnull,dflt_value,pk
0,0,Row ID,INTEGER,0,None,0
1,1,Order ID,TEXT,0,None,0
2,2,Order Date,TEXT,0,None,0
3,3,Ship Date,TEXT,0,None,0
4,4,Ship Mode,TEXT,0,None,0
5,5,Customer ID,TEXT,0,None,0
6,6,Customer Name,TEXT,0,None,0
7,7,Segment,TEXT,0,None,0
8,8,Country,TEXT,0,None,0
9,9,City,TEXT,0,None,0


<font size = '4' > Exploratory Queries </font>

In [8]:
query = "SELECT COUNT(*) FROM superstore;"
pd.read_sql_query(query, conn)

,COUNT(*)
0,9994


In [9]:
query ="SELECT * FROM superstore LIMIT 2;"
pd.read_sql_query(query, conn)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.0,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94,3,0.0,219.5820


In [13]:
query = """
SELECT 
    MIN(Sales) AS Min_Sales,
    MAX(Sales) AS Max_Sales,
    AVG(Sales) AS Avg_Sales,
    SUM(Sales) AS Total_Sales
FROM superstore;
"""
pd.read_sql_query(query, conn)

,Min_Sales,Max_Sales,Avg_Sales,Total_Sales
0,0.444,22638.48,229.858001,2.297201e+06


In [14]:
query = """
SELECT
    Region,
    COUNT(*) AS Count
FROM superstore
GROUP BY Region
ORDER BY Count DESC;
"""
pd.read_sql_query(query, conn)

,Region,Count
0,West,3203
1,East,2848
2,Central,2323
3,South,1620


In [16]:
query = """
SELECT 
    COUNT(DISTINCT City),
    COUNT(DISTINCT [Product Name]),
    COUNT(DISTINCT [Customer Name])
FROM superstore;
"""
pd.read_sql_query(query, conn)

,COUNT(DISTINCT City),COUNT(DISTINCT [Product Name]),COUNT(DISTINCT [Customer Name])
0,531,1850,793


<font size = '4'> Key Performance Indicators </font>

1. Total Sales and Profit by Region

In [18]:
query = """
SELECT
    Region,
    COUNT(DISTINCT [Customer Name]) AS Num_Customers,
    ROUND(SUM(Sales), 2) AS Total_Sales,
    ROUND(SUM(Profit), 2) AS Total_Profit,
    ROUND(SUM(Profit) / SUM(Sales) * 100, 2) AS Profit_Margin_Percent
FROM superstore
GROUP BY Region
ORDER BY Total_Sales DESC;
""" 
pd.read_sql_query(query, conn)

,Region,Num_Customers,Total_Sales,Total_Profit,Profit_Margin_Percent
0,West,686,725457.82,108418.45,14.94
1,East,674,678781.24,91522.78,13.48
2,Central,629,501239.89,39706.36,7.92
3,South,512,391721.91,46749.43,11.93


2. Top Selling Products (by revenue)
- Some top selling products by revenue generated negative profit

In [17]:
query = """
SELECT
    [Product Name],
    COUNT(*) AS Order_Count,
    ROUND(SUM(Sales), 2) AS Total_Sales,
    ROUND(SUM(Profit), 2) AS Total_Profit,
    ROUND(AVG(Sales), 2) AS Avg_Sale,
    ROUND(SUM(Profit) / NULLIF(SUM(Sales), 0) * 100, 2) AS Profit_Margin_Percent
FROM superstore
GROUP BY [Product Name]
HAVING COUNT(*) > 5
ORDER BY Total_Sales DESC
LIMIT 10;

""" 
pd.read_sql_query(query, conn)

,Product Name,Order_Count,Total_Sales,Total_Profit,Avg_Sale,Profit_Margin_Percent
0,Fellowes PB500 Electric Punch Plastic Comb Bin...,10,27453.38,7753.04,2745.34,28.24
1,HON 5400 Series Task Chairs for Big and Tall,8,21870.58,0.00,2733.82,0.00
2,GBC DocuBind TL300 Electric Binding System,11,19823.48,2233.51,1802.13,11.27
3,GBC Ibimaster 500 Manual ProClick Binding System,9,19024.50,760.98,2113.83,4.00
4,Hewlett Packard LaserJet 3310 Copier,8,18839.69,6983.88,2354.96,37.07
5,GBC DocuBind P400 Electric Binding System,6,17965.07,-1878.17,2994.18,-10.45
6,Martin Yale Chadless Opener Electric Letter Op...,6,16656.20,-1299.18,2776.03,-7.80
7,Samsung Galaxy Mega 6.3,6,13943.67,1822.76,2323.94,13.07
8,Apple iPhone 5,6,12996.60,1611.58,2166.10,12.40
9,Bretford Rectangular Conference Table Tops,12,12995.29,-327.23,1082.94,-2.52


3. Top Purchasing Customers

In [19]:
query = """
SELECT
    [Customer Name],
    COUNT(*) AS Order_Count,
    ROUND(SUM(Sales), 2) AS Total_Sales,
    ROUND(SUM(Profit), 2) AS Total_Profit,
    ROUND(AVG(Sales), 2) AS Avg_Sale_Value
FROM superstore
GROUP BY [Customer Name]
ORDER BY Total_Sales DESC
LIMIT 10;

""" 
pd.read_sql_query(query, conn)

,Customer Name,Order_Count,Total_Sales,Total_Profit,Avg_Sale_Value
0,Sean Miller,15,25043.05,-1980.74,1669.54
1,Tamara Chand,12,19052.22,8981.32,1587.68
2,Raymond Buch,18,15117.34,6976.10,839.85
3,Tom Ashbrook,10,14595.62,4703.79,1459.56
4,Adrian Barton,20,14473.57,5444.81,723.68
5,Ken Lonsdale,29,14175.23,806.85,488.80
6,Sanjit Chand,22,14142.33,5757.41,642.83
7,Hunter Lopez,11,12873.30,5622.43,1170.30
8,Sanjit Engle,19,12209.44,2650.68,642.60
9,Christopher Conant,11,12129.07,2177.05,1102.64


4. Top 5 Product Subcategories

In [20]:
query = """
SELECT
    [Sub-Category],
    ROUND(SUM(Sales), 2) AS Total_Sales,
    ROUND(SUM(Profit), 2) AS Total_Profit,
    ROUND(SUM(Profit) / SUM(Sales) * 100, 2) AS Profit_Margin_Percent,
    COUNT(DISTINCT [Customer Name]) AS Num_Customers
FROM superstore
GROUP BY [Sub-Category]
ORDER BY Total_Sales DESC
LIMIT 5;
""" 
pd.read_sql_query(query, conn)

,Sub-Category,Total_Sales,Total_Profit,Profit_Margin_Percent,Num_Customers
0,Phones,330007.05,44515.73,13.49,511
1,Chairs,328449.10,26590.17,8.10,407
2,Storage,223843.61,21278.83,9.51,514
3,Tables,206965.53,-17725.48,-8.56,261
4,Binders,203412.73,30221.76,14.86,650


Findings:
1. The west region generated the highest sales ($725k) and had the most profit ($108k)
2. Electric punches, chairs, and binding systems were the highest selling products (by revenue)
3. The phones subcateogry generated (330k) in total revenue, the highest of all subcategories.

Implications:
1. Prioritize the west and east regions for expansion given their high performance in sales and profit margins.
2. Evaluate pricing and supplier costs for products with strong sales but negative profitability
3. Implemented targeted retention strategies for high spending customers, such as Tamara Chand and Raymond Buch.
